# Criando Primitivas SQL-like

## Criando um Lista de Associação

A lista representa o esquema de um conjunto de dados.

In [243]:
(define prehistoric-schema
  '((name . 0) (type . 1) (period . 2) (size . 3) (diet . 4)))

In [244]:
(assq 'type prehistoric-schema)

(type . 1)

In [245]:
(cdr (assq 'period prehistoric-schema))

2

In [246]:
(list-ref '("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") 2)

"Cretaceous"

## Projeção de 1 elemento (`project-1`)

In [247]:
(define (project-1 row column-name)
  (let ((position (cdr (assq column-name prehistoric-schema))))
    (list-ref row position)))

In [248]:
(project-1 '("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") 'period)

"Cretaceous"

## Tarefa

O que vai acontecer no seguinte cenário?

In [249]:
(define (project-1 row column-name)
  (let ((position (cdr (assq 'column-name prehistoric-schema))))
    (list-ref 'row position)))

In [250]:
(project-1 ("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") period)

wrong-type-arg: (#f Wrong type to apply: ~S (Tyrannosaurus Rex) (Tyrannosaurus Rex))

## Macro `project-1`

### `define-macro`

In [251]:
(define-macro (project-1 row column-name)
  `(let ((position (cdr (assq ,column-name prehistoric-schema))))
     (list-ref ,row position)))

In [252]:
(project-1 '("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") 'period)

"Cretaceous"

In [253]:
(let ((position (cdr (assq 'period prehistoric-schema))))
    (list-ref '("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") position)))

"Cretaceous"

In [254]:
(define-macro (project-1 row column-name)
  `(let ((position (cdr (assq ',column-name prehistoric-schema))))
     (list-ref ',row position)))

In [255]:
(project-1 ("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") period)

"Cretaceous"

## Hygienic Macro `project-1`

### `define-syntax`

### Questão

Redefina a macro `project-1` usando `define-syntax` de forma que atenda a chamada a seguir.

In [256]:
(define-syntax project-1
 (syntax-rules ()
  [(project-1 row column-name)
   (let ((position (cdr (assq 'column-name prehistoric-schema))))
     (list-ref 'row position))]))

In [257]:
(project-1 ("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") period)

"Cretaceous"

### Questão

Usando `literals` modifique a macro `project-1` anterior de forma que atenda a chamada a seguir.

In [258]:
(define-syntax select-1
 (syntax-rules (from)
  [(select-1 column-name from row)
   (let ((position (cdr (assq 'column-name prehistoric-schema))))
     (list-ref 'row position))]))

In [259]:
(select-1 period from ("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore"))

"Cretaceous"

## Organizando o Conceito de Tabela

In [260]:
(define prehistoric-schema
  '(name type period size diet))

In [261]:
(define prehistoric-data
  '(("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore")
    ("Pteranodon"        "pterosaur" "Cretaceous" 7.0  "carnivore")
    ("Mosasaurus"        "aquatic"   "Cretaceous" 18.0 "carnivore")
    ("Stegosaurus"       "dinosaur"  "Jurassic"   9.0  "herbivore")
    ("Plesiosaurus"      "aquatic"   "Jurassic"   3.5  "carnivore")
    ("Triceratops"       "dinosaur"  "Cretaceous" 9.0  "herbivore")
    ("Quetzalcoatlus"    "pterosaur" "Cretaceous" 11.0 "carnivore")
    ("Ichthyosaurus"     "aquatic"   "Jurassic"   2.0  "carnivore")))

### Definindo `create-table`
* função de ordem superior que empacota esquema e dados e os recupera através da função retornada;
* o princípio de closure mantém a função atrelada aos dados.

In [262]:
(define (create-table schema data)
  (let ((table-schema schema)
        (indexed-schema (map cons schema (iota (length schema))))
        (table-data data))
    (define (dispatch part)
      (case part
        ((schema) table-schema)
        ((indexed-schema) indexed-schema)
        ((data) table-data)
        (else (error "Unknown part:" part))))
    dispatch))

In [263]:
(define prehistoric-table (create-table prehistoric-schema prehistoric-data))

In [264]:
(prehistoric-table 'schema)

(name type period size diet)

In [265]:
(prehistoric-table 'indexed-schema)

((name . 0) (type . 1) (period . 2) (size . 3) (diet . 4))

In [266]:
(prehistoric-table 'data)

(("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") ("Pteranodon" "pterosaur" "Cretaceous" 7.0 "carnivore") ("Mosasaurus" "aquatic" "Cretaceous" 18.0 "carnivore") ("Stegosaurus" "dinosaur" "Jurassic" 9.0 "herbivore") ("Plesiosaurus" "aquatic" "Jurassic" 3.5 "carnivore") ("Triceratops" "dinosaur" "Cretaceous" 9.0 "herbivore") ("Quetzalcoatlus" "pterosaur" "Cretaceous" 11.0 "carnivore") ("Ichthyosaurus" "aquatic" "Jurassic" 2.0 "carnivore"))

## Definindo a Operação de Projeção como uma Macro

In [267]:
(define-syntax projection
  (syntax-rules ()
    ((projection table columns)
      (let ((columns-proj
            (map (lambda (col)
                  (cdr (assq col (table 'indexed-schema))))
                              'columns)))
        (create-table 'columns
          (map (lambda (row)
                (map (lambda (col) (list-ref row col))
                      columns-proj))
              (table 'data)))))))

In [268]:
(define sub-table (projection prehistoric-table (name type)))

In [269]:
(sub-table 'schema)

(name type)

In [270]:
(sub-table 'indexed-schema)

((name . 0) (type . 1))

In [271]:
(sub-table 'data)

(("Tyrannosaurus Rex" "dinosaur") ("Pteranodon" "pterosaur") ("Mosasaurus" "aquatic") ("Stegosaurus" "dinosaur") ("Plesiosaurus" "aquatic") ("Triceratops" "dinosaur") ("Quetzalcoatlus" "pterosaur") ("Ichthyosaurus" "aquatic"))

## Definindo a Operação de Seleção como uma Macro

In [272]:
(define-syntax selection
  (syntax-rules ()
    ((selection table condition)
     (let ((schema (table 'schema))
           (data (table 'data)))
       (create-table schema
         (filter (lambda (row)
                   (apply (eval `(lambda ,schema condition) (interaction-environment)) row))
                 data))))))

In [273]:
(define large-creatures (selection prehistoric-table (> size 9)))

In [274]:
(large-creatures 'data)

(("Tyrannosaurus Rex" "dinosaur" "Cretaceous" 12.0 "carnivore") ("Mosasaurus" "aquatic" "Cretaceous" 18.0 "carnivore") ("Quetzalcoatlus" "pterosaur" "Cretaceous" 11.0 "carnivore"))

## Montando as Operações em uma operação de SELECT SQL-like

In [275]:
(define-syntax select
  (syntax-rules (from where)
    ((select columns from table where condition)
     (projection (selection table condition) columns))))

In [276]:
(define large-types (select (name type) from prehistoric-table where (> size 9)))

In [277]:
(large-types 'schema)

(name type)

In [278]:
(large-types 'data)

(("Tyrannosaurus Rex" "dinosaur") ("Mosasaurus" "aquatic") ("Quetzalcoatlus" "pterosaur"))